Build P3HT bulk simulation 

In [3]:
import os,sys, copy, pickle

In [4]:
from streamm import *

In [5]:
%load_ext autoreload
%autoreload 2

Load resources from previous example 

In [6]:
res_tag = 'local'  # Change this to remote to run the calculations remotely 
res_local = resource.Resource(res_tag)
res_local.load_json()

Reading in resource json file res_local.json 


In [7]:
peregrine = resource.Resource('peregrine')
peregrine.load_json()

Reading in resource json file res_peregrine.json 


Load oligomer from previous examples 

In [ ]:
p3ht_n10_esp = buildingblock.Container('p3ht_n10_esp')

In [ ]:
p3ht_n10_esp.read_cply()

In [ ]:
p3ht_n10_esp.lat_cubic(50.0)

In [ ]:
p3ht_n10_esp.guess_oplsa()

In [ ]:
p3ht_n10_esp.bonded_bonds()
p3ht_n10_esp.bonded_angles()
p3ht_n10_esp.bonded_dih()

Place the oligomer into a simulation and add parameters 

In [ ]:
lmp_oligo = lammps.LAMMPS('p3ht_n10_esp_nvt')
lmp_oligo.load_json()

In [ ]:
lmp_oligo.set_resource(peregrine)

In [ ]:
lmp_oligo.properties['scratch'] = lmp_oligo.dir['scratch']

In [ ]:
lmp_oligo.make_dir()

In [ ]:
lmp_oligo.add_strucC(p3ht_n10_esp)

In [ ]:
print lmp_oligo.strucC.lat._matrix

In [ ]:
lmp_oligo.read_param("conj.param")

In [ ]:
lmp_oligo.set_ffparam()

In [ ]:
for p,pt in lmp_oligo.strucC.particles.iteritems(): # properties["charge"] 
    print p,pt.properties["symbol"],pt.properties["fftype"]

Read in templates

In [ ]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_nvt.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_oligo.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_oligo.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
print lmp_oligo.dir['launch']

In [ ]:
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.load_str('templates','run')

In [ ]:
lmp_oligo.load_str('templates','in')

In [ ]:
lmp_oligo.write_data()

In [ ]:
lmp_oligo.replacewrite_prop('in','input','in','%s.in'%(lmp_oligo.tag))

In [ ]:
lmp_oligo.properties['input_in'] = lmp_oligo.files['input']['in']

In [ ]:
lmp_oligo.replacewrite_prop('run','scripts','run','%s.sh'%(lmp_oligo.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_oligo.tag)
lmp_oligo.add_file(file_type,file_key,file_name)

In [ ]:
os.chdir(lmp_oligo.dir['home'])
lmp_oligo.dump_json()
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.push()

In [ ]:
lmp_oligo.run()

In [ ]:
lmp_oligo.check()
print lmp_oligo.meta['status']

In [ ]:
lmp_oligo.analysis()
print lmp_oligo.meta['status']

In [ ]:
print lmp_oligo.files['output']

In [ ]:
os.chdir(lmp_oligo.dir['home'])
lmp_oligo.dump_json()
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.meta['status'] = 'finished'

In [ ]:
lmp_oligo.store()

In [ ]:
print lmp_oligo.files['output']

In [ ]:
lmp_oligo.pull()

In [ ]:
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.read_data(lmp_oligo.files['output']['data_3'])

In [ ]:
lmp_oligo.strucC.write_cply()

In [ ]:
lmp_oligo.strucC.group_prop('mol','mol')

In [ ]:
lmp_oligo.strucC.groupsets['mol'].group_pbcs()

In [ ]:
print lmp_oligo.strucC.tag 

In [ ]:
lmp_oligo.strucC.tag = "%s_nvt"%(lmp_oligo.strucC.tag )

In [ ]:
os.chdir(lmp_oligo.dir['home'])

In [ ]:
lmp_oligo.dump_json()

In [ ]:
lmp_oligo.strucC.write_xyz()

In [ ]:
lmp_oligo.strucC.write_cply()

In [ ]:
p3ht_n10_nvt = copy.deepcopy(lmp_oligo.strucC)

Replicate the oligomer into large simulation cell

In [ ]:
xO = 20

In [ ]:
p3ht_n10_esp_x = buildingblock.Container('p3ht_n10_x%d'%(xO))
p3ht_n10_esp_x.read_cply()

In [ ]:
p3ht_n10_esp_x.lat_cubic(150.0)

In [ ]:
print p3ht_n10_esp_x.lat._matrix

Set seed 

In [ ]:
s = 2984927

In [ ]:
p3ht_n10_esp_x = p3ht_n10_esp_x.add_struc(p3ht_n10_nvt,xO,s)

In [ ]:
p3ht_n10_esp_x.bonded_bonds()
p3ht_n10_esp_x.bonded_angles()
p3ht_n10_esp_x.bonded_dih()

In [ ]:
p3ht_n10_esp_x.tag = 'p3ht_n10_x%d'%(xO)

In [ ]:
p3ht_n10_esp_x.write_xyz()

In [ ]:
p3ht_n10_esp_x.write_cply()

In [ ]:
print p3ht_n10_esp_x.n_bonds,p3ht_n10_esp_x.n_angles,p3ht_n10_esp_x.n_dihedrals

Create LAMMPS calculation to equilibrate bulk system

In [8]:
xO = 20

In [9]:
lmp_bulk = lammps.LAMMPS('p3ht_n10_esp_x%d'%(xO))

In [10]:
lmp_bulk.load_json()

Resource tag found peregrine 
Reading in resource json file res_peregrine.json 


In [ ]:
lmp_bulk.set_resource(peregrine)

In [ ]:
lmp_bulk.properties['scratch'] = lmp_bulk.dir['scratch']

In [ ]:
lmp_bulk.make_dir()

In [ ]:
lmp_bulk.strucC = buildingblock.Container()

In [ ]:
lmp_bulk.add_strucC(p3ht_n10_esp_x)

In [ ]:
print lmp_bulk.strucC.n_particles

In [ ]:
lmp_bulk.read_param("conj.param")

In [ ]:
lmp_bulk.set_ffparam()

In [ ]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_nvt.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_bulk.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_bulk.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
print lmp_bulk.dir['launch']

In [ ]:
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.load_str('templates','run')

In [ ]:
lmp_bulk.load_str('templates','in')

In [ ]:
lmp_bulk.write_data()

In [ ]:
lmp_bulk.replacewrite_prop('in','input','in','%s.in'%(lmp_bulk.tag))

In [ ]:
lmp_bulk.properties['input_in'] = lmp_bulk.files['input']['in']

In [ ]:
lmp_bulk.replacewrite_prop('run','scripts','run','%s.sh'%(lmp_bulk.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_bulk.tag)
lmp_bulk.add_file(file_type,file_key,file_name)

In [ ]:
os.chdir(lmp_bulk.dir['home'])
lmp_bulk.dump_json()
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.push()

In [ ]:
lmp_bulk.run()

In [ ]:
lmp_bulk.check()
print lmp_bulk.meta['status']

In [ ]:
lmp_bulk.analysis()
print lmp_bulk.meta['status']

In [ ]:
print lmp_bulk.files['output']

In [ ]:
print lmp_bulk.dir

In [ ]:
os.chdir(lmp_bulk.dir['home'])
lmp_bulk.dump_json()
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.meta['status'] = 'finished'

In [ ]:
lmp_bulk.store()

In [ ]:
lmp_bulk.pull()

In [ ]:
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.read_data_pos(lmp_bulk.files['output']['data_3'])

In [ ]:
pos_nvt = copy.deepcopy(lmp_bulk.strucC.positions)

In [ ]:
lmp_bulk.strucC.write_cply()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.group_prop('mol','mol')

In [ ]:
lmp_bulk.strucC.groupsets['mol'].group_pbcs()

In [ ]:
print lmp_bulk.tag 

In [ ]:
lmp_bulk.strucC.tag = "%s_nvt"%(lmp_bulk.strucC.tag )

Write new data file and copy it to scratch to be stored with output, 
since LAMMPS does not print dihedral angles for opls style 

In [ ]:
data_file = '%s.data'%(lmp_bulk.tag)

In [ ]:
lmp_bulk.write_data(data_file)

In [ ]:
lmp_bulk.add_file('output','data_4',data_file)

In [ ]:
ssh_id = "%s@%s"%(lmp_bulk.resource.ssh['username'],lmp_bulk.resource.ssh['address'])
from_pathfile = os.path.join(lmp_bulk.dir['launch'],data_file)
bash_command = "scp %s %s:%s"%(from_pathfile,ssh_id,lmp_bulk.dir['scratch'])
print bash_command
os.system(bash_command)

In [ ]:
lmp_bulk.meta['status'] = 'finished'
print lmp_bulk.meta['status']

In [ ]:
lmp_bulk.store()

In [ ]:
os.chdir(lmp_bulk.dir['home'])

In [ ]:
lmp_bulk.dump_json()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.write_cply()

In [ ]:
lmp_bulk.write_data()

In [ ]:
print lmp_bulk.tag

Run Equilibration 

In [13]:
lmp_equ = lammps.LAMMPS('p3ht_n10_x20_equ')

In [14]:
lmp_equ.load_json()

Resource tag found peregrine 
Reading in resource json file res_peregrine.json 
No references found 


In [33]:
lmp_equ.set_resource(peregrine)

In [34]:
lmp_equ.properties['scratch'] = lmp_equ.dir['scratch']

In [35]:
lmp_equ.make_dir()

In [36]:
lmp_equ.add_refcalc(lmp_bulk)

In [37]:
lmp_equ.properties['nodes'] = 4
lmp_equ.properties['nproc'] = lmp_equ.properties['nodes']*lmp_equ.properties['ppn']
print lmp_equ.properties

{u'uncompress': u'tar -xzf ', u'scratch': u'/scratch/tkemper/p3ht_n10_x20_equ/', u'compress': u'tar -czf ', u'ppn': 24, u'finish_str': u'Loop time of', u'allocation': u'orgopv', u'run_list': u'', u'nproc': 96, u'input_in': u'p3ht_n10_x20_equ.in', u'walltime': 4, u'compress_sufix': u'tgz', u'data_file': u'p3ht_n10_esp_x20.data', u'feature': u'24core', u'queue': u'short', u'e-mail': u'travis.kemper@nrel.gov', u'exe_command': u'qsub ', u'nodes': 4, u'pmem': 1500, u'comp_key': u'compressed'}


In [38]:
lmp_equ.files['input']['data_file'] = lmp_bulk.files['output']['data_4']
print lmp_equ.files['input']['data_file']

p3ht_n10_esp_x20.data


In [39]:
lmp_equ.properties['data_file']  = lmp_equ.files['input']['data_file'] 

In [40]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_equ1.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_equ.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates in lammps_equ1.in templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/lammps_equ1.in to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ/lammps_equ1.in 


In [41]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_equ.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates run lammps_peregrine.pbs templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/lammps_peregrine.pbs to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ/lammps_peregrine.pbs 


In [42]:
print lmp_equ.dir['launch']

/Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ/


In [43]:
os.chdir(lmp_equ.dir['launch'])

In [44]:
lmp_equ.load_str('templates','run')

In [45]:
lmp_equ.load_str('templates','in')

In [46]:
lmp_equ.replacewrite_prop('in','input','in','%s.in'%(lmp_equ.tag))

In [47]:
lmp_equ.properties['input_in'] = lmp_equ.files['input']['in']

In [48]:
lmp_equ.replacewrite_prop('run','scripts','run','%s.pbs'%(lmp_equ.tag))

In [49]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_equ.tag)
lmp_equ.add_file(file_type,file_key,file_name)

In [50]:
os.chdir(lmp_equ.dir['home'])
lmp_equ.dump_json()
os.chdir(lmp_equ.dir['launch'])

{'files': {u'templates': {u'run': u'lammps_peregrine.pbs', u'in': u'lammps_equ1.in'}, u'input': {u'data_file': u'p3ht_n10_esp_x20.data', u'in': u'p3ht_n10_x20_equ.in'}, u'scripts': {u'run': u'p3ht_n10_x20_equ.pbs'}, u'data': {}, u'output': {u'log': u'p3ht_n10_x20_equ.log'}}, 'meta': {u'date': u'2016-11-08T11:43:25.026199', u'status': u'written', u'resource': 'peregrine', u'software': u'lammps'}, 'references': {'p3ht_n10_esp_x20': 'p3ht_n10_esp_x20'}, 'units': {u'distance': u'angstroms', u'energy': u'kcal/mol', u'angle': u'radians', u'time': u'ns'}, 'data': {}, 'properties': {u'uncompress': u'tar -xzf ', u'scratch': u'/scratch/tkemper/p3ht_n10_x20_equ/', u'compress': u'tar -czf ', u'ppn': 24, u'finish_str': u'Loop time of', u'allocation': u'orgopv', u'run_list': '', u'nproc': 96, u'input_in': u'p3ht_n10_x20_equ.in', u'walltime': 4, u'compress_sufix': u'tgz', u'data_file': u'p3ht_n10_esp_x20.data', u'feature': u'24core', u'queue': u'short', u'e-mail': u'travis.kemper@nrel.gov', u'exe_com

In [51]:
ref_calc = lmp_equ.references['p3ht_n10_esp_x20']

In [52]:
print ref_calc.files

{u'templates': {u'run': u'lammps_peregrine.pbs', u'in': u'lammps_nvt.in'}, u'input': {u'data_file': u'p3ht_n10_esp_x20.data', u'compressed': u'p3ht_n10_esp_x20_input.tgz', u'in': u'p3ht_n10_esp_x20.in'}, u'scripts': {u'run': u'p3ht_n10_esp_x20.sh', u'compressed': u'p3ht_n10_esp_x20_scripts.tgz'}, u'data': {}, u'output': {u'log': u'p3ht_n10_esp_x20.log', u'data_3': u'min2.data', u'data_2': u'nvt1.data', u'data_1': u'min1.data', u'compressed': u'p3ht_n10_esp_x20_output.tgz', u'data_4': u'p3ht_n10_esp_x20.data', u'dcd_1': u'nvt1_dump.dcd'}}


In [53]:
lmp_equ.push()

 Resource type ssh 
runnning push function in /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ 
Compressing and copying input files to scratch directory 
 file_i  p3ht_n10_esp_x20.data
 file_i  p3ht_n10_x20_equ.in
> compressed_file  p3ht_n10_x20_equ_input.tgz
Compressing and copying templates files to scratch directory 
 file_i  lammps_peregrine.pbs
 file_i  lammps_equ1.in
> compressed_file  p3ht_n10_x20_equ_templates.tgz
Compressing and copying scripts files to scratch directory 
 file_i  p3ht_n10_x20_equ.pbs
> compressed_file  p3ht_n10_x20_equ_scripts.tgz
Copying output of reference calculations p3ht_n10_esp_x20


In [54]:
lmp_equ.run()

Calculation with status written 
Resource type ssh 
Executing run command ssh tkemper@peregrine.hpc.nrel.gov ' cd /scratch/tkemper/p3ht_n10_x20_equ/ ; qsub p3ht_n10_x20_equ.pbs '  


In [ ]:
lmp_equ.check()
print lmp_equ.meta['status']

In [ ]:
lmp_bulk.analysis()
print lmp_bulk.meta['status']

In [ ]:
print lmp_bulk.files['output']

In [ ]:
os.chdir(lmp_bulk.dir['home'])
lmp_bulk.dump_json()
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.meta['status'] = 'finished'

In [ ]:
lmp_bulk.store()

In [ ]:
lmp_bulk.pull()

In [ ]:
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.read_data(lmp_bulk.files['output']['data_3'])

In [ ]:
lmp_bulk.strucC.write_cply()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.group_prop('mol','mol')

In [ ]:
lmp_bulk.strucC.groupsets['mol'].group_pbcs()

In [ ]:
print lmp_bulk.strucC.tag 

In [ ]:
lmp_bulk.strucC.tag = "%s_nvt"%(lmp_bulk.strucC.tag )

In [ ]:
os.chdir(lmp_bulk.dir['home'])

In [ ]:
lmp_equ.dump_json()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.write_cply()

Analyize bulk properties with streamm

In [11]:
import pandas as pd

In [15]:
print lmp_equ.tag

p3ht_n10_x20_equ


In [56]:
os.getcwd()

'/Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ_rdf1'

In [17]:
equ_rdf = calculation.CalculationRes('%s_rdf1'%(lmp_equ.tag))

In [18]:
equ_rdf.set_resource(peregrine)

In [19]:
equ_rdf.properties['scratch'] = equ_rdf.dir['scratch']

In [20]:
equ_rdf.add_refcalc(lmp_bulk)

In [100]:
print lmp_bulk.files['output']['dcd_1']

{u'log': u'p3ht_n10_esp_x20.log', u'data_3': u'min2.data', u'data_2': u'nvt1.data', u'data_1': u'min1.data', u'compressed': u'p3ht_n10_esp_x20_output.tgz', u'data_4': u'p3ht_n10_esp_x20.data', u'dcd_1': u'nvt1_dump.dcd'}


In [21]:
equ_rdf.make_dir()

In [55]:
os.chdir(equ_rdf.dir['launch'])

In [23]:
file_type = 'input'
file_key = 'cply'
file_name = "p3ht_n10_x20_nvt.cply"
from_dirkey = 'home'
to_dirkey = 'launch'
equ_rdf.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  input cply p3ht_n10_x20_nvt.cply home launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/examples/p3ht_n10_x20_nvt.cply to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ_rdf1/p3ht_n10_x20_nvt.cply 


In [24]:
p3ht_n10_x20_nvt = buildingblock.Container('p3ht_n10_x20_nvt')

In [25]:
p3ht_n10_x20_nvt.read_cply()

In [26]:
p3ht_n10_x20_nvt.group_prop('residue','residue')

In [28]:
gromacs_i  = gromacs.GROMACS(p3ht_n10_x20_nvt.tag)

In [29]:
gromacs_i.add_strucC(p3ht_n10_x20_nvt)

In [32]:
gromacs_i.write_gro()

Create pandas data frame of particle properties 

In [61]:
p3ht_n10_x20_nvt.propcompile_particles()

In [62]:
part_df =  pd.DataFrame(p3ht_n10_x20_nvt.prop_particles)

In [63]:
syCA = part_df['fftype'] == 'CA'
syS = part_df['fftype'] == 'S'

In [64]:
sub_i = part_df[ syCA | syS ]

In [65]:
struc_i = p3ht_n10_x20_nvt.getSubStructure(sub_i.index,'sel_SCA') 

In [66]:
struc_i.write_xyz()

In [67]:
syCT = part_df['fftype'] == 'CT'

In [68]:
sub_j = part_df[ syCT ]

In [69]:
struc_j = p3ht_n10_x20_nvt.getSubStructure(sub_j.index,'sel_CT') 

In [70]:
struc_j.write_xyz()

Check that selections are what you expect and save lists 

In [71]:
list_i = [str(pkey) for pkey in sub_i.index]

In [81]:
outfile = open('list_i','wb')
outfile.write("\n".join(list_i))
outfile.close()

In [73]:
equ_rdf.add_file('input','list_i','list_i')

In [74]:
list_j = [str(pkey) for pkey in sub_j.index]

In [ ]:
outfile = open('list_j','wb')
outfile.write("\n".join(list_j))
outfile.close()

In [ ]:
equ_rdf.add_file('input','list_j','list_j')

In [92]:
print equ_rdf.tag,equ_rdf.properties

p3ht_n10_x20_equ_rdf1 {u'queue': u'short', 'compress_sufix': 'tgz', 'uncompress': 'tar -xzf ', u'scratch': u'/scratch/tkemper/p3ht_n10_x20_equ_rdf1/', 'compress': 'tar -czf ', u'ppn': 24, u'feature': u'24core', u'allocation': u'orgopv', u'e-mail': u'travis.kemper@nrel.gov', 'comp_key': 'compressed', u'nproc': 24, u'nodes': 1, u'pmem': 1500, u'walltime': 4, u'exe_command': u'qsub '}


In [88]:
file_type = 'templates'
file_key = 'run'
file_name = "streamm_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
equ_rdf.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates run streamm_peregrine.pbs templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/streamm_peregrine.pbs to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ_rdf1/streamm_peregrine.pbs 


In [95]:
equ_rdf.dir['scripts'] = '/Users/tkemper/Development/streamm_ext/streamm-tools/scripts'
print equ_rdf.dir['scripts']

/Users/tkemper/Development/streamm_ext/streamm-tools/scripts


In [96]:
file_type = 'scripts'
file_key = 'py_script'
file_name = "rdf2.py"
from_dirkey = 'scripts'
to_dirkey = 'launch'
equ_rdf.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  scripts py_script rdf2.py scripts launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/scripts/rdf2.py to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ_rdf1/rdf2.py 


In [101]:
equ_rdf.properties['streamm_command'] ='python %s  %s --list_i list_i --list_j list_i --dcd %s '%(equ_rdf.files['scripts']['py_script'],equ_rdf.tag,lmp_bulk.files['output']['dcd_1'])
print equ_rdf.properties['streamm_command']

python rdf2.py  p3ht_n10_x20_equ_rdf1 --list_i list_i --list_j list_i --dcd nvt1_dump.dcd 


In [103]:
equ_rdf.load_str('templates','run')

In [104]:
equ_rdf.replacewrite_prop('run','scripts','run','%s.pbs'%(equ_rdf.tag))

In [105]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(equ_rdf.tag)
equ_rdf.add_file(file_type,file_key,file_name)

In [ ]:
equ_rdf.push()

 Resource type ssh 
runnning push function in /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ_rdf1 
Compressing and copying input files to scratch directory 
 file_i  p3ht_n10_x20_nvt.cply
 file_i  list_i
> compressed_file  p3ht_n10_x20_equ_rdf1_input.tgz
Compressing and copying templates files to scratch directory 
 file_i  streamm_peregrine.pbs
> compressed_file  p3ht_n10_x20_equ_rdf1_templates.tgz
Compressing and copying scripts files to scratch directory 
 file_i  p3ht_n10_x20_equ_rdf1.pbs
 file_i  rdf2.py
> compressed_file  p3ht_n10_x20_equ_rdf1_scripts.tgz
Copying output of reference calculations p3ht_n10_esp_x20


In [82]:
lfile = open('list_i','rb')
t_i = lfile.readlines()
lfile.close()
l_i = [int(pkey) for pkey in  t_i]

In [87]:
print l_i

[0, 1, 2, 3, 4, 26, 27, 28, 29, 30, 51, 52, 53, 54, 55, 76, 77, 78, 79, 80, 101, 102, 103, 104, 105, 126, 127, 128, 129, 130, 151, 152, 153, 154, 155, 176, 177, 178, 179, 180, 201, 202, 203, 204, 205, 226, 227, 228, 229, 230, 252, 253, 254, 255, 256, 278, 279, 280, 281, 282, 303, 304, 305, 306, 307, 328, 329, 330, 331, 332, 353, 354, 355, 356, 357, 378, 379, 380, 381, 382, 403, 404, 405, 406, 407, 428, 429, 430, 431, 432, 453, 454, 455, 456, 457, 478, 479, 480, 481, 482, 504, 505, 506, 507, 508, 530, 531, 532, 533, 534, 555, 556, 557, 558, 559, 580, 581, 582, 583, 584, 605, 606, 607, 608, 609, 630, 631, 632, 633, 634, 655, 656, 657, 658, 659, 680, 681, 682, 683, 684, 705, 706, 707, 708, 709, 730, 731, 732, 733, 734, 756, 757, 758, 759, 760, 782, 783, 784, 785, 786, 807, 808, 809, 810, 811, 832, 833, 834, 835, 836, 857, 858, 859, 860, 861, 882, 883, 884, 885, 886, 907, 908, 909, 910, 911, 932, 933, 934, 935, 936, 957, 958, 959, 960, 961, 982, 983, 984, 985, 986, 1008, 1009, 1010, 1011, 

Find pairs 

In [ ]:
pairs_ii = p3ht_n10_x20_nvt.find_pairs(sub_i.index,sub_i.index)

In [ ]:
pairs_ii_inter = p3ht_n10_x20_nvt.find_pairs(sub_i.index,sub_i.index,mol_inter=True)

In [ ]:
pairs_ii_intra = p3ht_n10_x20_nvt.find_pairs(sub_i.index,sub_i.index,mol_intra=True)

In [ ]:
print pairs_ii_intra